# Fake news prediction

importing the libraries




In [1]:
#importing the dependencies
import nltk
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing stopword in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

 data preprocessing

In [4]:
# loading the  dataset to pandasframe
fake = pd.read_csv(
    '/content/fake.csv',
    engine='python',
    on_bad_lines='skip'
)

true = pd.read_csv(
    '/content/true.csv',
    engine='python',
    on_bad_lines='skip'
)



In [5]:
# concatenating the fake and true csv files in one
fake['label']=1
true['label']=0
news_dataset=pd.concat([fake,true])

In [6]:
#shuffling the data
news_dataset=news_dataset.sample(frac=1)

In [7]:
#first five row of dataset
news_dataset.head()

,title,text,subject,date,label
282,Trump Ally Curt Schilling Blasts ‘Horrifying’...,"Former Red Sox pitcher Curt Schilling, the hos...",News,"October 1, 2017",1
4078,Trump's concession on wall funding removes obs...,WASHINGTON (Reuters) - The threat of a U.S. go...,politicsNews,"April 25, 2017",0
12939,Germany against Trump's Jerusalem decision: Me...,BERLIN (Reuters) - Germany does not support th...,worldnews,"December 6, 2017",0
19653,RIDICULOUS! NBC STIRS UP FEAR Of A Trump Presi...,During the first half of their extended post-e...,left-news,"Nov 10, 2016",1
21272,“WHITE STUDENT UNION” Groups Spring Up World-W...,Individual people can be bullied into submiss...,left-news,"Dec 1, 2015",1


In [8]:
#checking if there any null value exist ,if exist then filling it with empty string
news_dataset.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [9]:
#merging title and subject in one column called content
news_dataset['content']=news_dataset['title']+news_dataset['subject']

In [10]:
news_dataset.head()

,title,text,subject,date,label,content
282,Trump Ally Curt Schilling Blasts ‘Horrifying’...,"Former Red Sox pitcher Curt Schilling, the hos...",News,"October 1, 2017",1,Trump Ally Curt Schilling Blasts ‘Horrifying’...
4078,Trump's concession on wall funding removes obs...,WASHINGTON (Reuters) - The threat of a U.S. go...,politicsNews,"April 25, 2017",0,Trump's concession on wall funding removes obs...
12939,Germany against Trump's Jerusalem decision: Me...,BERLIN (Reuters) - Germany does not support th...,worldnews,"December 6, 2017",0,Germany against Trump's Jerusalem decision: Me...
19653,RIDICULOUS! NBC STIRS UP FEAR Of A Trump Presi...,During the first half of their extended post-e...,left-news,"Nov 10, 2016",1,RIDICULOUS! NBC STIRS UP FEAR Of A Trump Presi...
21272,“WHITE STUDENT UNION” Groups Spring Up World-W...,Individual people can be bullied into submiss...,left-news,"Dec 1, 2015",1,“WHITE STUDENT UNION” Groups Spring Up World-W...


In [11]:
print(news_dataset['content'])

282       Trump Ally Curt Schilling Blasts ‘Horrifying’...
4078     Trump's concession on wall funding removes obs...
12939    Germany against Trump's Jerusalem decision: Me...
19653    RIDICULOUS! NBC STIRS UP FEAR Of A Trump Presi...
21272    “WHITE STUDENT UNION” Groups Spring Up World-W...
                               ...                        
12611    Venezuela's Maduro says government wins 90 per...
16260    German CSU hopes for coalition by year's end d...
6849     Factbox: Trump fills top jobs for his administ...
7720     Oil industry workers split between Trump and C...
6957      Secret Documents Reveal Connection Between Pl...
Name: content, Length: 44898, dtype: object


In [12]:
#separating the data and label
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [13]:
print(X)

                                                   title  \
282     Trump Ally Curt Schilling Blasts ‘Horrifying’...   
4078   Trump's concession on wall funding removes obs...   
12939  Germany against Trump's Jerusalem decision: Me...   
19653  RIDICULOUS! NBC STIRS UP FEAR Of A Trump Presi...   
21272  “WHITE STUDENT UNION” Groups Spring Up World-W...   
...                                                  ...   
12611  Venezuela's Maduro says government wins 90 per...   
16260  German CSU hopes for coalition by year's end d...   
6849   Factbox: Trump fills top jobs for his administ...   
7720   Oil industry workers split between Trump and C...   
6957    Secret Documents Reveal Connection Between Pl...   

                                                    text       subject  \
282    Former Red Sox pitcher Curt Schilling, the hos...          News   
4078   WASHINGTON (Reuters) - The threat of a U.S. go...  politicsNews   
12939  BERLIN (Reuters) - Germany does not support th... 

In [14]:
print(Y)

282      1
4078     0
12939    0
19653    1
21272    1
        ..
12611    0
16260    0
6849     0
7720     0
6957     1
Name: label, Length: 44898, dtype: int64


Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act


In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [18]:
print(news_dataset['content'])

282      trump alli curt schill blast horrifi recoveri ...
4078     trump concess wall fund remov obstacl spend ta...
12939         germani trump jerusalem decis merkelworldnew
19653    ridicul nbc stir fear trump presid zero credib...
21272    white student union group spring world wide me...
                               ...                        
12611    venezuela maduro say govern win percent mayors...
16260    german csu hope coalit year end despit tricki ...
6849     factbox trump fill top job administrationpolit...
7720     oil industri worker split trump clintonpolitic...
6957     secret document reveal connect plan parenthood...
Name: content, Length: 44898, dtype: object


In [19]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [20]:
print(X)

['trump alli curt schill blast horrifi recoveri effort puerto rico video news'
 'trump concess wall fund remov obstacl spend talkspoliticsnew'
 'germani trump jerusalem decis merkelworldnew' ...
 'factbox trump fill top job administrationpoliticsnew'
 'oil industri worker split trump clintonpoliticsnew'
 'secret document reveal connect plan parenthood terrorist anti abort movementnew']


In [21]:
print(Y)

[1 0 0 ... 0 0 1]


In [22]:
Y.shape

(44898,)

In [23]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 433436 stored elements and shape (44898, 23833)>
  Coords	Values
  (0, 652)	0.28059771657318994
  (0, 2228)	0.2563288290152497
  (0, 5166)	0.4008867571720752
  (0, 6694)	0.27621291198503944
  (0, 9979)	0.3627533465664378
  (0, 14109)	0.09848390176278239
  (0, 16619)	0.2639165329454885
  (0, 17164)	0.3916001580935706
  (0, 17833)	0.26994649845631774
  (0, 18524)	0.4008867571720752
  (0, 21805)	0.09064254440164973
  (0, 22704)	0.11165851312624026
  (1, 4377)	0.4591143229780672
  (1, 8441)	0.28514111971542816
  (1, 14470)	0.4681859650957156
  (1, 17426)	0.3272158135010245
  (1, 19778)	0.3372329715441375
  (1, 20803)	0.4017612805052853
  (1, 21805)	0.108369637377106
  (1, 22945)	0.30782805565948296
  (2, 5472)	0.44887752220483307
  (2, 8676)	0.4246552282115347
  (2, 11112)	0.4507125364375545
  (2, 13155)	0.6261735990996417
  (2, 21805)	0.151465311211797
  :	:
  (44894, 20804)	0.3348130220890568
  (44894, 21733)	0.439637374427634

Splitting the dataset to training & test data

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [26]:
model = LogisticRegression()

In [27]:
model.fit(X_train, Y_train)

LogisticRegression()

Accuracy

In [28]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [29]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9768361267331143


In [30]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)


In [31]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9625835189309577


Making a predictive system

In [32]:
X_new=X_test[4]#first row  in test data
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The news is real')
else:
  print('The news is fake')

[0]
The news is real


In [33]:
print(Y_test[4])# original

0
